In [2]:
from utils import *
from train import *
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def show_image(img, ax=None):
    if ax is None:
        ax = plt
    if len(img.shape) == 2:
        ax.imshow(img, cmap='gray')
    else:
        ax.imshow(img)

In [ ]:
#model = training('large-data/vehicles', 'large-data/non-vehicles')
model = Model('model.pkl')

In [ ]:
test_image = read_image('test_images/test2.jpg')

search = CarSearch(model)
annotated = search.annotate_cars_in_image(test_image)
show_image(annotated)

## Video

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process_video(video_filename):
    model = Model('model.pkl')
    search = CarSearch(model) 
    output = "annotated_P5_" + video_filename
    input_clip = VideoFileClip(video_filename)
    clip = input_clip.fl_image(search.annotate_cars_in_video)
    %time clip.write_videofile(output, audio=False)

v1 = 'test_video.mp4'
v2 = 'project_video.mp4'

process_video(v2)

[MoviePy] >>>> Building video annotated_P5_project_video.mp4
[MoviePy] Writing video annotated_P5_project_video.mp4


 76%|███████▌  | 953/1261 [06:39<02:21,  2.18it/s]